In [1]:
# tips: https://stackoverflow.com/questions/23317458/how-to-remove-punctuation
# copy right from TensorFlow team
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
# with customization

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile

In [2]:
import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer

In [ ]:
def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    local_filename = os.path.join(gettempdir(), filename)
    if not os.path.exists(local_filename):
        local_filename, _ = urllib.request.urlretrieve(url + filename,
                                                       local_filename)
    statinfo = os.stat(local_filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + local_filename +
                        '. Can you get to it with a browser?')
    return local_filename

In [4]:
filename = './BOT/en_US.news.txt'

In [21]:
# Read the data, strip ending '\n' and remove puntuations. Tokenize into a list of strings.
# Calculate running time of this function.
from time import time
def read_data(filename, remove_stopwords = True):
    """Extract the first file enclosed in a zip file as a list of words."""
    with open(filename,'r+',encoding="utf-8") as f:
        lines = f.readlines()
        data = []
        toker = RegexpTokenizer(r'\w+')
        for line in lines:
            line = line.strip().lower()
            data.extend(toker.tokenize(line))
        if remove_stopwords:
            stop_words = set(stopwords.words('english'))
            data = list(filter(lambda x: x not in stop_words, data))
    return data

start = time()
vocabulary = read_data(filename)
end = time()
print ('Time takes for running processing is: ', end-start)

print('Data size', len(vocabulary))

Time takes for running processing is:  48.128000020980835
Data size 20548326


Collect most frequent words from the data, give index to each word and create reverse dictionary.

In [10]:
import pickle

In [11]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000

def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [22]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])
d = {'data': data, 'count': count, 'dictionary': dictionary, 'reverse_dictionary': reverse_dictionary}

with open('news.pickle', 'wb') as f:
    pickle.dump(d, f, protocol = pickle.HIGHEST_PROTOCOL)

Most common words (+UNK) [['UNK', 482519], ('said', 250432), ('one', 89358), ('year', 81464), ('would', 72234)]
Sample data [36, 1276, 1869, 71, 256, 791, 313, 4, 2657, 58] ['home', 'alone', 'apparently', 'st', 'louis', 'plant', 'close', 'would', 'die', 'old']


In [25]:
data_index = 0

In [26]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
        buffer.extend(data[data_index:data_index + span])
        data_index += span
        for i in range(batch_size // num_skips):
            context_words = [w for w in range(span) if w != skip_window]
            words_to_use = random.sample(context_words, num_skips)
            for j, context_word in enumerate(words_to_use):
                batch[i * num_skips + j] = buffer[skip_window]
                labels[i * num_skips + j, 0] = buffer[context_word]
                if data_index == len(data):
                    buffer[:] = data[:span]
                    data_index = span
                else:
                    buffer.append(data[data_index])
                    data_index += 1
                    # Backtrack a little bit to avoid skipping words in the end of a batch
                    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [30]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3632 donated -> 3608 baseman
0 UNK -> 0 UNK


KeyError: 502934064

In [33]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.
num_sampled = 64      # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)


graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
      # Look up embeddings for inputs.
      embeddings = tf.Variable(
      tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
      embed = tf.nn.embedding_lookup(embeddings, train_inputs)

      # Construct the variables for the NCE loss
      nce_weights = tf.Variable(
      tf.truncated_normal([vocabulary_size, embedding_size],
      stddev=1.0 / math.sqrt(embedding_size)))
      nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

      # Compute the average NCE loss for the batch.
      # tf.nce_loss automatically draws a new sample of the negative labels each
      # time we evaluate the loss.
      # Explanation of the meaning of NCE loss:
      #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
      loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
      biases=nce_biases,
      labels=train_labels,
      inputs=embed,
      num_sampled=num_sampled,
      num_classes=vocabulary_size))

      # Construct the SGD optimizer using a learning rate of 1.0.
      optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

      # Compute the cosine similarity between minibatch examples and all embeddings.
      norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
      normalized_embeddings = embeddings / norm
      valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
      similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

      # Add variable initializer.
      init = tf.global_variables_initializer()

      # Step 5: Begin training.
      num_steps = 100001

      with tf.Session(graph=graph) as session:
          # We must initialize all variables before we use them.
          init.run()
          print('Initialized')

          average_loss = 0
          for step in xrange(num_steps):
              batch_inputs, batch_labels = generate_batch(
              batch_size, num_skips, skip_window)
              feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

              # We perform one update step by evaluating the optimizer op (including it
              # in the list of returned values for session.run()
              _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
              average_loss += loss_val

              if step % 2000 == 0:
                  if step > 0:
                      average_loss /= 2000
                      # The average loss is an estimate of the loss over the last 2000 batches.
                      print('Average loss at step ', step, ': ', average_loss)
                      average_loss = 0

                      # Note that this is expensive (~20% slowdown if computed every 500 steps)
                      if step % 10000 == 0:
                          sim = similarity.eval()
                          for i in xrange(valid_size):
                              valid_word = reverse_dictionary[valid_examples[i]]
                              top_k = 8  # number of nearest neighbors
                              nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                              log_str = 'Nearest to %s:' % valid_word
                              for k in xrange(top_k):
                                  close_word = reverse_dictionary[nearest[k]]
                                  log_str = '%s %s,' % (log_str, close_word)
                                  print(log_str)
                                  final_embeddings = normalized_embeddings.eval()


Initialized


InvalidArgumentError: indices[0] = 74252287 is not in [0, 50000)
	 [[Node: embedding_lookup = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@Variable"], validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable/read, _arg_Placeholder_0_0)]]

Caused by op 'embedding_lookup', defined at:
  File "C:\Anaconda3_install2\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3_install2\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3_install2\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3_install2\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3_install2\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Anaconda3_install2\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    supe